In [27]:
import os

import pandas as pd

from transformers import AutoTokenizer, AutoModel
import torch

import time
from joblib import Parallel, delayed

from tqdm import tqdm

In [6]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [10]:
# Load FLORES dataset and convert to Pandas DataFrame
flores_plus_dev = load_dataset("openlanguagedata/flores_plus", split='dev').to_pandas()
flores_plus_dev = flores_plus_dev.rename(columns={'iso_15924': 'language'})

Resolving data files:   0%|          | 0/218 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/213 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/218 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/213 [00:00<?, ?it/s]

ace_Arab.parquet:   0%|          | 0.00/125k [00:00<?, ?B/s]

...

zsm_Latn.parquet:   0%|          | 0.00/119k [00:00<?, ?B/s]

yor_Latn.parquet:   0%|          | 0.00/135k [00:00<?, ?B/s]

zul_Latn.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating dev split:   0%|          | 0/217346 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/215556 [00:00<?, ? examples/s]

In [11]:
# Clean and prepare text data
def clean_texts(ids, texts):
    cleaned_ids = []
    cleaned_texts = []
    for id_, text in zip(ids, texts):
        if isinstance(text, str) and text.strip():
            cleaned_ids.append(id_)
            cleaned_texts.append(text)
    return cleaned_ids, cleaned_texts

In [12]:
# Function to compute fertility for a single text
def fertility(text, tokenizer):
    tokenized = tokenizer.tokenize(text)
    num_words = len(text.split())
    fertility_score = len(tokenized) / num_words if num_words > 0 else 0
    return fertility_score, tokenized

In [13]:
# Parallel processing function
def parallel_fertility_batches(texts, tokenizer, batch_size=32, n_jobs=-1):
    def process_batch(batch):
        return [fertility(text, tokenizer) for text in batch]
    
    # Split texts into batches
    batches = [texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_batch)(batch) for batch in batches
    )
    
    # Flatten results
    fertility_scores = []
    tokenized_texts = []
    for batch_results in results:
        for score, tokens in batch_results:
            fertility_scores.append(score)
            tokenized_texts.append(tokens)
    return fertility_scores, tokenized_texts

In [19]:
# Function to load cached data if available
def load_cached_results(csv_name):
    if os.path.exists(csv_name):
        print(f"Loading cached results from {csv_name}")
        return pd.read_csv(csv_name)
    return None

# Function to save intermediate results
def save_results_to_cache(dataframe, csv_name):
    dataframe.to_csv(csv_name, index=False)
    print(f"Cached results saved to {csv_name}")

# Function to process fertility
def process_model(model_name, name_for_csv, tokenizer, ids, texts):
    # Clean and prepare text data
    cleaned_ids, cleaned_texts = clean_texts(ids, texts)

    # Compute fertility scores and tokenized texts
    fertility_scores, tokenized_texts = parallel_fertility_batches(
        cleaned_texts, tokenizer, batch_size=32, n_jobs=-1
    )

    # Create DataFrame
    output_df = pd.DataFrame({
        'id': cleaned_ids,
        'text': cleaned_texts,
        'fertility_score': fertility_scores,
        'tokens': tokenized_texts  # Keep tokens as a list
    })
    return output_df

In [26]:
# Check if GPU is available
print("GPU Available:", torch.cuda.is_available())

# If GPU is available, show GPU details
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
else:
    print("Using CPU.")


GPU Available: False
Using CPU.


In [23]:
# List of models for tokenization
models = [
    {"name": "google/flan-t5-xxl", "csv_name": "flan-t5-xxl"},
    {"name": "bigscience/mt0-xxl", "csv_name": "mt0-xxl"},
    {"name": "bigscience/bloom-7b1", "csv_name": "bloom-7b"},
    {"name": "meta-llama/Llama-2-7b", "csv_name": "llama-2-7b"},
    {"name": "bert-base-multilingual-cased", "csv_name": "mBERT"},
    {"name": "QwQ-32B-Preview", "csv_name": "qwq-32b-preview"}
]

In [ ]:
# Start processing
start_time = time.time()

# Add progress bar for models
print("Processing models:")
for model_info in tqdm(models, desc="Model Progress", unit="model"):
    model_name = model_info["name"]
    name_for_csv = f"flores_fertilized_and_tokenized_with_{model_info['csv_name']}.csv"
    
    # Check if results are cached
    cached_results = load_cached_results(name_for_csv)
    if cached_results is not None:
        tqdm.write(f"Skipped {model_name} (cached results found).")
        continue

    try:
        # Load tokenizer
        tqdm.write(f"Loading tokenizer for {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
        model = AutoModel.from_pretrained(model_name).to("cuda")
    except OSError as e:
        tqdm.write(f"Error loading model {model_name}: {e}")
        continue

    # Clean and prepare text and ID data
    ids = flores_plus_dev['id'].tolist()
    raw_texts = flores_plus_dev['text'].tolist()
    tqdm.write(f"Processing {model_name}...")
    output_df = process_model(model_name, name_for_csv, tokenizer, ids, raw_texts)
    
    # Save results to cache
    tqdm.write(f"Saving results for {model_name}...")
    save_results_to_cache(output_df, name_for_csv)

end_time = time.time()
print(f"Runtime: {end_time - start_time:.2f} seconds")

Processing models:


Model Progress:   0%|          | 0/6 [00:00<?, ?model/s]

Loading cached results from flores_fertilized_and_tokenized_with_flan-t5-xxl.csv


Model Progress:  17%|█▋        | 1/6 [00:02<00:13,  2.64s/model]

Skipped google/flan-t5-xxl (cached results found).
Loading tokenizer for bigscience/mt0-xxl...


Model Progress:  17%|█▋        | 1/6 [00:04<00:13,  2.64s/model]

Processing bigscience/mt0-xxl...


Model Progress:  17%|█▋        | 1/6 [04:40<23:21, 280.34s/model]


KeyboardInterrupt: 